In [1]:
import glob 
import pandas as pd 
import xml.etree.ElementTree as ET 
from datetime import datetime
import requests
from zipfile import ZipFile

In [2]:
def get_data():
    print('Iniciar descarga de archivos')
    # Define the remote file to retrieve
    remote_url = 'http://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBMDeveloperSkillsNetwork-PY0221EN-SkillsNetwork/labs/module%206/Lab%20-%20Extract%20Transform%20Load/data/datasource.zip'
    # Define the local filename to save data
    local_file = 'datasource.zip'
    # Make http request for remote file data
    data = requests.get(remote_url)
    # Save file data to local copy
    with open(local_file, 'wb') as f:
        f.write(data.content)

    with ZipFile(local_file, 'r') as zipObj:
        # Extract all the contents of zip file in different directory
        zipObj.extractall('dealership_data')
    
    tmpfile    = "dealership_temp.tmp"
    logfile    = "dealership_logfile.txt"            # all event logs will be stored
    targetfile = "dealership_transformed_data.csv"   # transformed data is stored

In [3]:
def extract_from_csv(file_to_process): 
    dataframe = pd.read_csv(file_to_process) 
    return dataframe

In [4]:
def extract_from_json(file_to_process):
    dataframe = pd.read_json(file_to_process, lines=True)
    return dataframe

In [5]:
def extract_from_xml(file_to_process):
    dataframe = pd.DataFrame(columns=['car_model','year_of_manufacture','price', 'fuel'])
    tree = ET.parse(file_to_process) 
    root = tree.getroot() 
    for person in root: 
        car_model = person.find("car_model").text 
        year_of_manufacture = int(person.find("year_of_manufacture").text)
        price = float(person.find("price").text) 
        fuel = person.find("fuel").text 
        dataframe = dataframe.append({"car_model":car_model, "year_of_manufacture":year_of_manufacture, "price":price, "fuel":fuel}, ignore_index=True) 
    return dataframe

In [6]:
def extract():
    extracted_data = pd.DataFrame(columns=['car_model','year_of_manufacture','price', 'fuel']) 
    #for csv files
    for csvfile in glob.glob("dealership_data/*.csv"):
        extracted_data = extracted_data.append(extract_from_csv(csvfile), ignore_index=True)
    #for json files
    for jsonfile in glob.glob("dealership_data/*.json"):
        extracted_data = extracted_data.append(extract_from_json(jsonfile), ignore_index=True)
    return extracted_data
    #for xml files
    for xmlfile in glob.glob("dealership_data/*.xml"):
        extracted_data = extracted_data.append(extract_from_xml(xmlfile), ignore_index=True)
    return extracted_data

In [7]:
def transform(data):
    data['price'] = round(data.price, 2)
    return data

In [8]:
def load(targetfile, data_to_load):
    data_to_load.to_csv(targetfile)

In [10]:
def log(message):
    timestamp_format = '%H:%M:%S-%h-%d-%Y'
    #Hour-Minute-Second-MonthName-Day-Year
    now = datetime.now() # get current timestamp
    timestamp = now.strftime(timestamp_format)
    with open("dealership logfile.txt","a") as f: 
        f.write('[' + timestamp + ']: ' + message + '\n')
        print(message)

In [13]:
if __name__ == '__main__':
    targetfile = "dealership_transformed_data.csv"   # transformed data is stored
    log("ETL Job Started")
    
    get_data()
    log("Extract phase Started")
    extracted_data = extract() 
    print(extracted_data)
    log("Extract phase Ended")

    log("Transform phase Started")
    transformed_data = transform(extracted_data)
    print(transformed_data)
    log("Transform phase Ended")

    log("Load phase Started")
    load(targetfile, transformed_data)
    log("Load phase Ended")

    log("ETL Job Ended")

ETL Job Started
Iniciar descarga de archivos
Extract phase Started
        car_model year_of_manufacture         price    fuel
0        alto k10                2016   4253.731343  Petrol
1           ignis                2017   7313.432836  Petrol
2             sx4                2011   6567.164179  Petrol
3        alto k10                2014   3731.343284  Petrol
4         wagon r                2013   4328.358209  Petrol
5           swift                2011   4477.611940  Petrol
6           swift                2013   6194.029851  Petrol
7           swift                2017   8955.223881  Petrol
8        alto k10                2010   2910.447761  Petrol
9            ciaz                2015  11119.402985  Diesel
10           ritz                2014   5000.000000  Petrol
11            sx4                2013   7089.552239  Diesel
12           ciaz                2017  10820.895522  Petrol
13        wagon r                2011   4253.731343  Petrol
14          swift                